In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.2 Safari/605.1.15"
        }

In [3]:
icd_chap_url=u'http://code.nhsa.gov.cn:8000/jbzd/public/toStdOperationTreeList.html'
icd_chapter=requests.get(icd_chap_url,headers=headers)

In [4]:
icd_chapter=icd_chapter.json()

In [6]:
icd_chapter[0]['operationId']

'044d7011-7979-11e9-9611-8cec4bd010f3'

In [7]:
icd_detail_url_base=u'http://code.nhsa.gov.cn:8000/jbzd/public/dataOperationSearchDetail.html?operationId='

In [8]:
icd_detail_url_full=icd_detail_url_base+icd_chapter[0]['operationId']

In [9]:
icd_detail_url_full

'http://code.nhsa.gov.cn:8000/jbzd/public/dataOperationSearchDetail.html?operationId=044d7011-7979-11e9-9611-8cec4bd010f3'

In [10]:
icd_detail=requests.get(icd_detail_url_full,headers=headers)

In [11]:
if icd_detail.status_code == 200:
    info = {}
    soup = BeautifulSoup(icd_detail.text)
    icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
    icd_name_lst=[d.text.strip() for d in soup.find_all('span')]


In [15]:
CHS_ICD9=[]
for i in range(len(icd_chapter)):
    icd_detail_url_full=icd_detail_url_base+icd_chapter[i]['operationId']
    icd_detail=requests.get(icd_detail_url_full,headers=headers)
    if icd_detail.status_code == 200:
        info = {}
        soup = BeautifulSoup(icd_detail.text)
        icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
        icd_name_lst=[d.text.strip() for d in soup.find_all('span')]
        df_icd=pd.DataFrame({'icd_code':icd_code_lst,'icd_name':icd_name_lst})
        CHS_ICD9.append(df_icd)
    else:
        print(icd_detail_url_full)

In [16]:
CHS_ICD9_Table=pd.concat(CHS_ICD9,axis=0,ignore_index=True)

In [17]:
CHS_ICD9_Table.head(20)

,icd_code,icd_name
0,00.0,治疗性超声
1,00.01,头和颈部血管治疗性超声
2,00.0100,头和颈部血管治疗性超声
3,00.0101,头部血管治疗性超声
4,00.0102,颈部血管治疗性超声
5,00.02,心脏治疗性超声
6,00.0200,心脏治疗性超声
7,00.0200x001,心脏血管治疗性超声
8,00.03,周围血管治疗性超声
9,00.0300,周围血管治疗性超声


In [18]:
len(CHS_ICD9_Table)

17552

In [19]:
CHS_ICD9_Table.to_csv('./LOOKUP_TABLES/CHS_ICD9_CM3.csv',encoding='utf-8',index=False)